

Homework, coding assignment 1.2: 1) Convert all your variables to fully dynamic external input 2) Add variables for a) returns b) 401k matching rate c) regular salary increases d) other variables as needed to incorporate these into your loop e) account for inflation if appropriate f) expected retirement target g) (Extra) calculate your expected retirement costs

a) use 10yr mean return on the S&P500(SPY) for your rate of return b) retire at age 67 c) max out contributions every year, on a biweekly basis d) 401k matching e) use 2% annual inflation


In [18]:
import numpy as np
import yfinance as yf

age = int(input('What is your current age? '))
money = int(input('How much do you need to retire? (Enter a whole number) '))
cost = float(input('What do is your current living expense? (biweekly) '))
inflation = 1+float(input('What is the expected inflation rate in decimals? (Enter 0.02 for 2% expected inflation) '))
salary = float(input('What is your salary biweekly? '))
salary_growth = float(input('Are you expecting a salary growth? (Enter a percetage i.e. 10% or 10)').replace('%',''))/100
if input('Does your 401K match your contribution? (Y/N)').lower() == 'y':
    matching = float(input('By how much? (Enter a percentage i.e. 10% or 10)').replace('%',''))/100
else:
    matching = 0
contribution = float(input('How much do you plan to put in biweekly? '))
if input('Do you have an initial contributions or a current savings? (Y/N) ').lower() == 'y':
    balance = float(input('Please enter your current balance: '))
else:
    balance = 0
    
#I will use daily rate of return adjusted for closed market days for calculation, since contribution is biweekly
#I understand that each year has 252 trading days, but this 365 day return is implemented to help with other calculations
#And the annualized return should stay the same either way
if input('Do you have an annual rate of return in mind? (Y/N) ').lower() == 'y':
    rate_of_return = float(input('Please enter your expected rate of return: ').replace('%',''))/100
    daily_return = (1+rate_of_return)**(1/365)-1
else:
    print('S&P500 10-year return will be used.')
    spy = yf.download('SPY', period='10y')
    begin = spy['Adj Close'][0]
    end = spy['Adj Close'][-1]
    daily_return = (1+(end-begin)/begin)**(1/3650)-1
    print(daily_return)
retire_age = int(input('Final question: when do you want to retire? '))

#Setting initial values
day = 0
life_expectancy = 100
inflation_rate = 1
trigger = 0
extra_day_over_year = 0
while balance <money or age < retire_age:

    #assuming first contribution is made on day 0 the first time and every 14 day since, calculation is ignoring leap year
    if day == 365:
        age +=1
        day = 0
        extra_day_over_year += 1
        if extra_day_over_year==7:
            extra_day_over_year = 0
        #Salary growth and contribution grows with salary
        salary = salary * (1+salary_growth)
        contribution = contribution * (1+salary_growth)
        inflation_rate = inflation_rate * inflation

    #Calculate wealth accumulation on a daily and bi-weekly basis
    balance = balance * (1+daily_return)
    if (day+extra_day_over_year)%14 == 0:
        balance = balance + contribution*(1+matching)

    #In case you don't have enough money to retire                 
    if age > retire_age and trigger == 0:
        trigger += 1
        print('Unforunately, you won\'t be able to retire at '+ str(retire_age)+'...')
        print('At your retirement age, you have accummulated $'+str(balance)+', and you are $'+str(money-balance)+'short.')
    day += 1
    if age > life_expectancy:
        print('And you really should rethink your strategy... you are not going to retire at age of 100!')
        break

    #This means retirement can happen
    if balance > money and trigger == 0 and age < retire_age:
        temp_age = age
        temp_balance = balance
        trigger += 1
        print('You can retire early at '+str(temp_age)+' with $'+str(temp_balance)+', but you can keep working.')
    #This means if the person keeps working

#print outcome for each cases    
if age <= retire_age:
    print('You can retire at '+str(age)+' with $'+str(balance)+', good work!')
elif age >= life_expectancy:
    print('Maybe you should consider lowering your target, or try to make more.')
    trigger = 2
else: #age > retire age
    print('You can retire at '+str(age)+' with $'+str(balance)+', but at what cost?')

#Analyzing costs
if input('Press c for cost...').lower() == 'c' and trigger != 2:
    if input('Do you plan to put your money in a savings account? (Y/N)').lower() == 'y':
        rate_of_return = float(input('what is savings account annual rate? (Enter a percentage, i.e. 1% or 1)').replace('%',''))
        daily_return = (1+rate_of_return)**(1/365)-1
        trigger = 3 #for savings account case, since savings account won't crash
    cost = cost * inflation_rate
    day = 0
    year = 0
    while balance > 0:
        if day == 365:
            year +=1
            day = 0
            extra_day_over_year += 1
            if extra_day_over_year==7:
                extra_day_over_year = 0
            cost = cost * inflation
        day+=1
        balance = balance * (1+daily_return)
        if day%14 == 0:
            balance = balance - cost
        if year+age >= 100:
            if trigger == 3:
                print('You don\'t need to worry about living expenses until age of 100 if you save this much!')
            else:
                print('You don\'t need to worry about cost until age of 100 unless market crashes big time.')
            break
    if year+age < 100:
        print('You should be fine until '+str(age+year)+'.')
else:
    print('Have a good day!')

What is your current age? 27
How much do you need to retire? (Enter a whole number) 10000000
What do is your current living expense? (biweekly) 1
What is the expected inflation rate in decimals? (Enter 0.02 for 2% expected inflation) 0.025
What is your salary biweekly? 1500
Are you expecting a salary growth? (Enter a percetage i.e. 10% or 10)10
Does your 401K match your contribution? (Y/N)n
How much do you plan to put in biweekly? 1000
Do you have an initial contributions or a current savings? (Y/N) n
Do you have an annual rate of return in mind? (Y/N) n
S&P500 10-year return will be used.
[*********************100%***********************]  1 of 1 completed
0.00033273239384223885
Final question: when do you want to retire? 67
You can retire early at 52 with $10001559.557152055, but you can keep working.
You can retire at 67 with $79614788.22670835, good work!
Press c for cost...c
Do you plan to put your money in a savings account? (Y/N)y
what is savings account annual rate? (Enter a pe